In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score
import xgboost as xgb

## Random Forest

Collecte et traitement des données

In [2]:
# Collecte des données
X = pd.read_csv(r"..\X_train.csv",index_col=0)
Y = pd.read_csv(r"..\Y_train.csv",index_col=0)

## Missing Data
data =  pd.merge(X, Y, left_index=True, right_index=True)
data = data.dropna(axis=1, thresh=len(data)*0.25)
data = data.dropna()
#data = data.ffill()
X = data[[col for col in X.columns if col in data.columns]]
Y = data[Y.columns]
## Format
Y=Y.spot_id_delta.to_numpy()
X = pd.get_dummies(X)
## Split
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42)

Creation et Validation du modèle

In [3]:
scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]

for scaler in scalers:
    # Appliquer le scaler
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Initialiser le modèle
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

    # Entraîner le modèle de base
    model.fit(X_train_scaled, y_train)
    
    # Évaluation du modèle
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Scaler: {scaler.__class__.__name__}")
    print(f"Mean Squared Error: {mse}")
    print(f"R^2 Score: {r2}")
    
    # Validation croisée
    cv_scores = sk.model_selection.cross_val_score(model, X_train_scaled, y_train, cv=10, scoring='neg_mean_squared_error')
    print(f"Cross-Validation Scores (neg MSE): {cv_scores}")
    print(f"Mean Cross-Validation Score (neg MSE): {cv_scores.mean()}\n")


Scaler: StandardScaler
Mean Squared Error: 372.15231756291354
R^2 Score: 0.5549331129978301
Cross-Validation Scores (neg MSE): [-3352.81977728  -352.12689452  -364.66549483  -389.49640988
  -459.01616027  -359.34499444  -394.7424122   -424.01814411
  -651.24295005 -2194.61409455]
Mean Cross-Validation Score (neg MSE): -894.2087332121158

Scaler: MinMaxScaler
Mean Squared Error: 372.15231756291354
R^2 Score: 0.5549331129978301
Cross-Validation Scores (neg MSE): [-3352.81977728  -352.12689452  -364.66549483  -389.49640988
  -459.01616027  -359.34499444  -394.7424122   -424.01814411
  -651.24295005 -2194.61409455]
Mean Cross-Validation Score (neg MSE): -894.2087332121158

Scaler: RobustScaler
Mean Squared Error: 372.15231756291354
R^2 Score: 0.5549331129978301
Cross-Validation Scores (neg MSE): [-3352.81977728  -352.12689452  -364.66549483  -389.49640988
  -459.01616027  -359.34499444  -394.7424122   -424.01814411
  -651.24295005 -2194.61409455]
Mean Cross-Validation Score (neg MSE): -894

Optimisation des paramètres

In [10]:
param_grid = {
    'n_estimators': [50,100, 200],
    'max_depth': [3, 4, 5, 6,10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
# Recherche en grille avec validation croisée
grid_search = sk.model_selection.GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_


Fitting 5 folds for each of 405 candidates, totalling 2025 fits


In [11]:
y_pred_best = best_model.predict(X_test_scaled)
best_mse = mean_squared_error(y_test, y_pred_best)
best_r2 = r2_score(y_test, y_pred_best)

print(f"Best Mean Squared Error: {best_mse}")
print(f"Best R^2 Score: {best_r2}")

best_cv_scores = sk.model_selection.cross_val_score(best_model, X, Y, cv=5, scoring='neg_mean_squared_error')
print(f"Best Cross-Validation Scores (neg MSE): {best_cv_scores}")
print(f"Mean Best Cross-Validation Score (neg MSE): {best_cv_scores.mean()}")

Best Mean Squared Error: 1812.7864474974865
Best R^2 Score: -1.1679596845478057
Best Cross-Validation Scores (neg MSE): [-1061.6675927  -2846.23514486 -2190.99988647  -895.48701446
  -438.74521667]
Mean Best Cross-Validation Score (neg MSE): -1486.6269710329818
